In [1]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import re
from scipy.signal import argrelmin, argrelmax
from bisect import bisect_right
from scipy.interpolate import UnivariateSpline
from scipy.stats import mode
import time

#Plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15, 10)
import seaborn as sns
pd.options.mode.chained_assignment = None

In [2]:
def processData(game_id):
    os.chdir("C:\Users\ckchiruka\Desktop\Current Courses\STA 160")
    motion=pd.read_csv("CSV Data/00"+str(game_id)+".csv",low_memory=False)
    motion=motion[motion.isnull().any(axis=1)==False].reset_index(drop=True)
    return(motion)

def getPlayRanges(motion):
    start = 0
    end = 0
    inplay = []
    no_dups = motion.drop_duplicates(subset = ['quarter', 'sec_in_quarter', 'shot_clock'], keep = 'last')
    nd_index = no_dups.index
    for i in range(max(nd_index)):
        if i not in nd_index:
            start = i
        else:
            inplay.append(start+1)
    
        if i in nd_index:
            end = i  
        else:
            inplay.append(end)

    inplay = sorted(list(set(inplay)))
    inplay = [i for i in inplay if i + 1 in nd_index or i - 1 in nd_index]

    if len(inplay) % 2 == 1:
        inplay.append(len(motion)-1)
    
    inplay = np.asarray(inplay).reshape((len(inplay)/2,2))

    minima = getRelMinima(motion)
    
    for i in range(len(inplay)):
        try:
            inplay[i][0] = minima[bisect_right(minima, inplay[i][0])-1]
        except IndexError:
            pass
        try: 
            inplay[i][1] = minima[bisect_right(minima, inplay[i][1])]
        except IndexError:
            inplay[i][1] = len(motion) - 1
        
    return inplay


def getRelMinima(motion):
    minima = list(argrelmin(motion['ball_z'].rolling(5, center = True).mean().as_matrix()))
    minima = [i for i in minima[0]]
    indices = np.where(np.diff(minima)<3)
    indices = [i for i in indices[0]]
    for index in sorted(indices, reverse=True):
        del minima[index]
    return minima

def getRelMaxima(motion):
    maxima = list(argrelmax(motion['ball_z'].rolling(5, center = True).mean().as_matrix()))
    maxima = [i for i in maxima[0]]
    indices = np.where(np.diff(maxima)<3)
    indices = [i for i in indices[0]]
    for index in sorted(indices, reverse=True):
        del maxima[index]
    return maxima

def deleteNotInPlay(motion, ranges):
    temp1 = pd.DataFrame()
    for i in ranges:
        temp = motion.iloc[i[0]:i[1]]
        temp = [temp1, temp]
        temp1 = pd.concat(temp, ignore_index = True)
    return temp1.reset_index(drop=True)

In [3]:
def NBA_to_SportVU(game):
    #Convert NBA API coordinates to SportVU coordinates
    Y=game.LOC_X/10+25
    Y_star=np.array(game.SIDE=="Left")*(50-Y)+np.array(game.SIDE=="Right")*Y
    X=game.LOC_Y/10+5
    X_star=np.array(game.SIDE=="Left")*X+np.array(game.SIDE=="Right")*(94-X)
    time = game.MINUTES_REMAINING*60 + game.SECONDS_REMAINING
    quarter = game.PERIOD
    player_id = game.PLAYER_ID
    side = game.SIDE
    return(np.array([X_star,Y_star, time, quarter, player_id, side]).T)

def sidingAlgorithm(game):
    #Find our candidate shot
    left_shots=np.where(game.SHOT_ZONE_AREA=="Left Side(L)")
    the_shot=game.iloc[np.min(left_shots),:]
    
    #Determine the team that took the shot
    shot_team=the_shot.TEAM_NAME
    
    #Set indicators based on shot.
    if the_shot.LOC_X>0:
        side_indicator=np.where(game.TEAM_NAME==shot_team,1,0)
    else:
        side_indicator=np.where(game.TEAM_NAME==shot_team,0,1)
        
    #Flip indicator for 2nd half.
    side_indicator[np.where(game.PERIOD>2)[0]]=1-side_indicator[np.where(game.PERIOD>2)[0]]
    left_right_map=np.vectorize(lambda x: "Left" if x == 1 else "Right")
    sides_labeled=left_right_map(side_indicator)
    
    #Set the sides dataframe to the shot dataframe
    shot_df=pd.DataFrame(sides_labeled,columns=["SIDE"])
    game_with_sides=pd.concat([game,shot_df],axis=1)
    return(game_with_sides)

def makeSecs(game):
    num_qtrs=np.max(game.PERIOD)
    secs=game.MINUTES_REMAINING*60+game.SECONDS_REMAINING+(num_qtrs-game.PERIOD)*720
    return(secs)

In [21]:
def makeGameShotsXY(game_id):
    playershotsdf=pd.read_csv("playershotsdf.csv")
    gameshots = playershotsdf[playershotsdf.GAME_ID == 00+game_id ].reset_index(drop=True)
    gameshots['sec_in_game'] = makeSecs(gameshots)
    gameshots = gameshots.sort_values('sec_in_game', ascending = False).reset_index()
    gameshots = sidingAlgorithm(gameshots)
    gameshotsXY = NBA_to_SportVU(gameshots)
    gameshotsXY = pd.DataFrame(gameshotsXY, columns = ['X', 'Y', 'sec_in_quarter', 'quarter', 'PLAYER_ID', 'SIDE'])
    return (gameshotsXY, gameshots)

def makeVoteSpace (motionXY, gameshotsXY):
    shots = []
    remove = []
    count = 0
    for i in gameshotsXY.as_matrix():
        balls = pd.DataFrame()
        players = pd.DataFrame()
        tempMotion = motionXY[(motionXY['quarter'] == i[3]) & (motionXY['sec_in_quarter'] >= i[2]+0.01) & 
                                (motionXY['sec_in_quarter'] <= i[2] + 4)]
        try:
            player_id = np.where(tempMotion.isin([i[4]]))[1][1]
            players['x'] = tempMotion.iloc[:, player_id+1]
            players['y'] = tempMotion.iloc[:, player_id+2]
            balls['x'] = tempMotion['ball_x']
            balls['y'] = tempMotion['ball_y']
            game = np.zeros(2)
            game[0] = i[0]
            game[1] = i[1]
            if i[5] == 'Left':
                side = 5
            else:
                side = 89
            ballx = tempMotion['ball_x']
            bally = tempMotion['ball_y']
            gradientx = np.diff(ballx)
            gradienty = np.diff(bally)
            gradient_slope = [gradienty[j]/gradientx[j] for j in range(len(gradientx))]
            gradient_intercept = [bally.iloc[j] - gradient_slope[j]*ballx.iloc[j] for j in range(len(gradientx))]
            y = [gradient_slope[j]*side + gradient_intercept[j] for j in range(len(gradientx))]
            y = [0] + y
            players = players.as_matrix()
            balls = balls.as_matrix()
            distance = [np.linalg.norm(players[j] - game) + np.linalg.norm(balls[j] - game) + (y[j] - 25)**2 for j in range(len(players))]
            ind = np.argpartition(distance, 5)[0:10]
            ind = [tempMotion.iloc[j].name for j in ind]
            shots.append(ind)
        except (IndexError, ValueError):
            shots.append([0,1,2,3,4,5,6,7,8,9])
            remove.append(count)
        count = count + 1
    
    for i in sorted(remove, reverse=True):
        del shots[i]
    
    return (shots, remove)

def findShots (motion, shots):
    minima = getRelMinima(motion)
    shot_parabola = []
    for i in shots:
        votespace = []
        for j in i:
            start = bisect_right(minima, j)-1
            votespace.append(start)
        start = mode(votespace)[0][0]
        try:
            end = minima[start+1]
        except IndexError:
            end = len(motion) - 1
        if start-1 < 0:
            start = 0
        else:
            start = minima[start]
        shot_parabola.append([start, end])
    
    return shot_parabola

def removeShots(shot_parabolas, gameshots, motion):
    remove = []
    count = 0
    for i in shot_parabolas:
        maxZ = max(motion['ball_z'].iloc[i[0]:i[1]])
        if maxZ < 7:
            remove.append(count)
        count = count + 1

    gameshots = gameshots.drop(gameshots.index[remove]).reset_index(drop = True)
    for i in sorted(remove, reverse=True):
            del shot_parabolas[i]
    
    return (shot_parabolas, gameshots)

def makeShotsDF(motion, gameshots, shot_parabolas):
    allballx = []
    allballz = []
    allbally = []
    predictor = []
    count = 0
    for i in shot_parabolas:
        ballz = motion['ball_z'].iloc[i[0]:i[1]]
        allballz.append(np.interp(np.linspace(0,len(ballz), 50),np.arange(len(ballz)), ballz))
        bally = motion['ball_y'].iloc[i[0]:i[1]]
        allbally.append(np.interp(np.linspace(0,len(bally), 50),np.arange(len(bally)), bally))
        ballx = motion['ball_x'].iloc[i[0]:i[1]]
        ballx = np.absolute(47 - ballx)
        allballx.append(np.interp(np.linspace(0,len(ballx), 50),np.arange(len(ballx)), ballx))
        predictor.append([gameshots['SHOT_MADE_FLAG'].iloc[count],gameshots['ACTION_TYPE'].iloc[count]])
        count = count + 1
    xcol = ['x' + str(i) for i in range(50)]
    ycol = ['y' + str(i) for i in range(50)]
    zcol = ['z' + str(i) for i in range(50)]
    return pd.concat([pd.DataFrame(allballx, columns = xcol), pd.DataFrame(allbally, columns = ycol), 
                       pd.DataFrame(allballz, columns = zcol), pd.DataFrame(predictor, columns = ['outcome', 'type'])], axis = 1 ) 

In [22]:
shotsdf = pd.DataFrame()
count = 0
for i in os.listdir(os.curdir+'/CSV Data'):
    game_id = int(i.replace('.csv', '').replace('00', '', 1)) 
    motion = processData(game_id)  
    ranges = getPlayRanges(motion)
    motion = deleteNotInPlay(motion, ranges)
    gameshotsXY, gameshots = makeGameShotsXY(game_id)
    gameshots['LOC_X'] = gameshotsXY['X']
    gameshots['LOC_Y'] = gameshotsXY['Y']
    shots, remove = makeVoteSpace(motion, gameshotsXY)
    gameshots = gameshots.drop(gameshots.index[remove]).reset_index(drop = True)
    gameshots = gameshots.drop('index', 1)
    shot_parabolas = findShots(motion, shots)
    shot_parabolas, gameshots = removeShots(shot_parabolas, gameshots, motion)
    shotsdf = pd.concat([shotsdf, makeShotsDF(motion, gameshots, shot_parabolas)])
    count = count + 1
    print count

C:\Users\ckchiruka\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ckchiruka\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\ckchiruka\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\ckchiruka\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [23]:
shotsdf = shotsdf.reset_index(drop=True)
shotsdf.to_csv('shotsdf.csv')

In [24]:
shotsdf

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z42,z43,z44,z45,z46,z47,z48,z49,outcome,type
0,38.91083,39.275498,39.612173,39.910089,40.117897,40.294453,40.430979,40.405784,40.321912,40.409149,...,9.310377,9.083292,8.784768,8.540022,8.352234,8.363356,8.368020,8.36802,0.0,Driving Layup Shot
1,36.15186,35.745863,35.349259,35.397803,35.740906,36.062488,36.191774,36.234641,36.279048,36.221180,...,6.581560,5.781117,5.110176,4.170382,3.292598,1.913550,0.714680,0.51535,1.0,Step Back Jump shot
2,35.88254,36.044548,36.280652,36.341317,36.385764,36.357227,36.155418,35.678599,35.212561,35.146876,...,4.861341,4.342865,3.730440,3.171583,2.685872,2.240070,1.891372,1.78924,1.0,Step Back Jump shot
3,34.25234,34.213698,34.418136,34.736369,35.096080,35.285298,35.307181,35.253593,35.249968,35.294532,...,4.833229,4.562255,4.415094,4.246170,4.311478,4.260138,3.590676,3.44074,1.0,Driving Floating Jump Shot
4,28.75305,28.739938,28.707292,28.805815,28.867704,28.910098,28.881723,28.597534,28.370406,28.146072,...,13.297020,12.779552,12.281024,11.774702,11.379511,10.631809,10.538980,10.53898,0.0,Jump Shot
5,37.20972,37.218120,37.226520,37.177628,37.081861,36.973156,36.748017,36.522879,36.310673,36.105433,...,3.672419,3.231643,2.790868,2.206166,1.605472,1.275090,1.275090,1.27509,0.0,Turnaround Hook Shot
6,40.51197,41.120059,41.569382,41.825807,41.651711,41.363097,41.261308,41.316389,41.458864,41.621107,...,6.686700,5.994590,5.184394,4.263677,2.993462,1.795566,0.735484,0.50119,1.0,Turnaround Fadeaway Bank Jump Shot
7,22.91343,23.066018,23.194471,23.295571,23.445645,23.645590,23.913652,24.079069,24.259786,24.497709,...,11.680657,11.905532,12.162590,12.126501,11.917972,12.131438,12.231500,12.23150,0.0,Pullup Jump shot
8,17.24620,17.493640,18.293533,18.282380,17.768583,17.299482,17.240103,17.747830,18.810865,20.064109,...,5.650744,5.111540,4.771720,4.599588,4.766646,4.643738,4.231831,4.33258,1.0,Running Pull-Up Jump Shot
9,27.76624,27.840308,28.136951,28.071653,27.790394,27.392218,26.837456,26.894770,26.931547,26.979862,...,6.953220,6.032475,5.550005,4.720760,4.047135,3.902048,3.801276,3.75822,0.0,Jump Shot
